In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import EmbeddingsModel
from trainer import EmbeddingTrainer, all_subjects
from pytorch_metric_learning import distances

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [2]:
subjects_test = [all_subjects[0]]
subjects_train = [sub for sub in all_subjects if sub not in subjects_test]

### W and B

In [3]:
import wandb
!wandb login

wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


In [4]:
sweep_config = {
    'method': 'random',
    "early_terminate": {
          "type": "hyperband",
          "max_iter": 10,
          "s": 2
    },
    "metric": {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "lr_steps": {"values": [2, 4, 8]},
        "batch_size": {"values": [128, 256, 512]}
    }
}

In [5]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        trainer_hyperparameter = {
            "path_train": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
            "subjects_train": subjects_train,
            "path_test": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
            "subjects_test": subjects_test,
            "acc_tester_metric": "AMI",
            "wandb": True,
            "acc_in_loop": False,
            "learning_rate": 0.001,
            "batch_size": 256,
            "margin": 0.09,
            "lr_steps": config.lr_steps,
            "filter": None
        }

        model_hyperparameter = {
            "sigmoid": False,
            "dropout": 0.35,
            "layers": [207, 512, 64]
        }

        distance  = distances.LpDistance(p=1)

        model = EmbeddingsModel(model_hyperparameter)        
        trainer = EmbeddingTrainer(trainer_hyperparameter, model, distance=distance, device="cuda:0")
        trainer.trainloop(10)

In [6]:
sweep_id = wandb.sweep(sweep_config, project="PainLevelShiftDetection")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: w39utkho
Sweep URL: https://wandb.ai/svenk/PainLevelShiftDetection/sweeps/w39utkho


In [7]:
wandb.agent(sweep_id, train, count=30)

wandb: Agent Starting Run: zbv95bpa with config:
wandb: 	batch_size: 128
wandb: 	lr_steps: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


wandb: Network error (ConnectionError), entering retry loop.


accuracy,▃▁▅▃▃▅█▇▃▇
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.87288
epoch,10


wandb: Agent Starting Run: 5occokjt with config:
wandb: 	batch_size: 128
wandb: 	lr_steps: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
